# Scryfall Price Trend Harvester

This program extracts magic the gathering (mtg) card information and prices from the Scryfall rest-API.

The idea is to get the bulk data in order to be able to store the all cards information (more info than the API will return in a normal query). If you need help understanding the data structure or you would like to query only for specific info, then look the Jupyter Notebook call "mtg_cards_query.ipynb". In there is an example of how to do that.

In [1]:
#Import libreries
import requests
import pandas as pd
from datetime import datetime

import sqlite3

### Setup

In [2]:
DEBUG = False

#Current date
now = datetime.now()
date_time = str(now.strftime("%Y%m%d"))

#pd.set_option('display.max_colwidth', None)

### Getting Bulk Data

Downloading the full card json that is generated every day.
More info about it in: https://scryfall.com/docs/api/bulk-data

In [3]:
api_response = requests.get('https://api.scryfall.com/bulk-data/default-cards')
print(api_response.status_code) #should be 200=ok

200


In [4]:
#Translate the response object's content from bytes to dictionary object so 
#it can be easily manipulate it
api_json = api_response.json() #converts from bytes to dictionary

#[Debug]
if DEBUG:
    print(api_json['download_uri'])

In [6]:
#Get cards information from bulk data in the responsed url.
bulk_url_json = requests.get(api_json['download_uri'], allow_redirects=True)

##Save data as json file in disk
#file_size = open('default_cards.json', 'wb').write(bulk_url_json.content)
#[Debug]
#if DEBUG:
    #print("Json downloaded correctly. File size: " + str(file_size))

##Save data to pandas
#Load json data to a pandas Data frame.
url_json = bulk_url_json.json() #converts from bytes to dictionary
cards_bulk_df = pd.DataFrame(url_json)

#[Debug]
if DEBUG:
    cards_bulk_df.to_csv('initial_bulk.csv', sep=";", index=False) 

#[Debug]
if DEBUG:
    #All columns types and info
    cards_bulk_df.info(verbose=True)
    #Summary of the first 3 rows
    print(cards_bulk_df.head(3))

#### Prices column 
(a dictionary of prices)

Ejemplo: {'usd': '76.45', 'usd_foil': '81.20', 'usd_etched': None, 'eur': '38.64', 'eur_foil': '43.93', 'tix': '1.12'}

In [7]:
#[Debug]
if DEBUG:
    print(cards_bulk_df['prices'].head(3))

### Save Data to local DB

First we need to filter the data that we don't want and keep the rest.

In my case is not only about the columns but also about the card editions. I'm an old school Magic collector so I love the old frames/border cards. I'll limit the db to that.

In [9]:
#colum = id #card global id
#colum = name #card name
#colum = image_uris #dictionary of card images
#colum = mana_cost #mana cost with color code
#colum = cmc "converted mana cost
#colum = power
#colum = toughness
#colum = reserved #if it's on the reserved list or not
#colum = foil
#colum = nonfoil
#colum = set_id #set global id
#colum = set_name #set name
#colum = rarity
#colum = frame #frame year (example: 1993, 1997, 2003, 2015, etc)
#colum = prices
columns_list = ['id', 'name', 'image_uris', 'mana_cost', 'cmc', 'power',
                'toughness', 'reserved', 'foil', 'nonfoil', 'set_id',
                'set_name', 'rarity', 'frame', 'prices', 'date_time']

#General Cleaning
#---------------------------------------------------
#Discard any card that's not in english
cards_df_flt = cards_bulk_df.loc[cards_bulk_df["lang"] == "en"]
#Discard the cards without image
cards_df_flt = cards_df_flt.dropna(subset = ['image_uris'])
#Adds date_time to the Data frame
cards_df_flt['date_time'] = date_time
#Keep only necessary columns
cards_df_flt = cards_df_flt[columns_list]

#[Debug]
if DEBUG:
    #All columns types and info
    cards_df_flt.info(verbose=True)

Now we split the columns with dictionary-like objects to obtain a "plain" table and store it on the DB.

In [10]:
df_split_images = pd.DataFrame(cards_df_flt['image_uris'].values.tolist())
#Keep only two of the images options, discard the rest.
df_split_images = df_split_images[['small', 'normal']]

df_split_prices = pd.DataFrame(cards_df_flt['prices'].values.tolist())

cards_df_flt = pd.concat([cards_df_flt.drop(['image_uris', 'prices'], axis=1), 
                          df_split_images, 
                          df_split_prices], 
                         axis=1)

cards_df_flt.to_csv('test.csv', sep=";", index=False)

#### Connect to the DB

In [11]:
#Generates (or creates if it doesn't exist) the db connection.
conn = sqlite3.connect("sqlite_db/mtg_cards.db")

#### Load Data frame to table

In [12]:
#Load pandas Data frame into scryfall_cards table.
cards_df_flt.to_sql(name="scryfall_cards", con=conn, if_exists="append", index=False)

67483

#### Testing result

In [15]:
#SQL query for testing
sql_result = conn.execute(
    """ SELECT
            id,
            name,
            set_name,
            usd
        FROM scryfall_cards
        WHERE
            set_name = 'Tempest'
            AND
            date_time = '20220417'
        ORDER BY id ASC
;""")

#Obtains the table column names because sqlite query returns only the data.
colums_names = [column[0] for column in sql_result.description]
#Creates a pandas dataframe qith the query data and the column names.
sql_df= pd.DataFrame.from_records(data = sql_result.fetchall(), columns = colums_names)

print(sql_df.head(5))

                                     id             name set_name    usd
0  000366c8-7a43-49d7-a103-ac5bd7efd9aa            Swamp  Tempest   0.34
1  012049f8-0936-49ed-948d-0d34af28550f  Armored Pegasus  Tempest   3.13
2  0169e52b-7909-4a8f-8ca2-62f030f9a85a   Canyon Wildcat  Tempest  26.47
3  0374f269-b07e-43af-911a-5454b35f14e6         Scabland  Tempest   0.28
4  03aa58b4-dbc2-414e-aa7a-f09360d59b3c    Winged Sliver  Tempest   0.03


#### Close connection to DB

In [16]:
#Close connection
conn.close()